# Random Under Sampling Method

This notebook will apply the random under sampling balancing method to the training data.

In [1]:

from google.colab import files


uploaded = files.upload()

Saving train.csv to train.csv
Saving val.csv to val.csv


In [2]:
import pandas as pd

df_train = pd.read_csv('train.csv')
X_train = df_train.drop('Diabetes_binary', axis=1)
y_train = df_train['Diabetes_binary']

val_df = pd.read_csv('val.csv')
X_val = val_df.drop('Diabetes_binary', axis=1)
y_val = val_df['Diabetes_binary']

print("Pre-sampling class imbalance:")
print(y_train.value_counts(normalize=True))

Pre-sampling class imbalance:
0.0    0.860664
1.0    0.139336
Name: Diabetes_binary, dtype: float64


In [3]:
from imblearn.under_sampling import RandomUnderSampler

random_under = RandomUnderSampler(random_state=92)
X_random_under, y_random_under = random_under.fit_resample(X_train, y_train)

print("Class distribution after Random under sampling:")
print(y_random_under.value_counts(normalize=True))

Class distribution after Random under sampling:
0.0    0.5
1.0    0.5
Name: Diabetes_binary, dtype: float64


# MLP #
## Hyperparameter Optimization ##

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [5]:
mlp_gs = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(10,), (20,), (50,), (100,), (150,)],
    'activation': ['logistic', 'tanh', 'relu'],
}
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_random_under, y_random_under) # X is train samples and y is the corresponding labels

GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=100), n_jobs=-1,
             param_grid={'activation': ['logistic', 'tanh', 'relu'],
                         'hidden_layer_sizes': [(10,), (20,), (50,), (100,),
                                                (150,)]})

In [6]:
print("Best parameters found:\n", clf.best_params_)

Best parameters found:
 {'activation': 'relu', 'hidden_layer_sizes': (100,)}


In [7]:
y_pred = clf.predict(X_val)
print(f"results on validation set: {classification_report(y_val, y_pred)}")

results on validation set:               precision    recall  f1-score   support

         0.0       0.95      0.69      0.80     43667
         1.0       0.29      0.78      0.42      7069

    accuracy                           0.70     50736
   macro avg       0.62      0.74      0.61     50736
weighted avg       0.86      0.70      0.75     50736

